In [ ]:
!nvidia-smi

In [ ]:
# Installation Library

!pip install transformers datasets sentencepiece rouge 

In [ ]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import LEDForConditionalGeneration, LEDTokenizer

In [ ]:
torch.manual_seed(0)

In [ ]:
dataset = load_dataset("billsum")

Using custom data configuration default


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
test_cases =  dataset['ca_test']

In [ ]:
case1 = test_cases[1]['text']
sum1 =  test_cases[1]['summary']

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


tokenizer = LEDTokenizer.from_pretrained("allenai/led-large-16384-arxiv")
model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv", return_dict_in_generate=True).to(device)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
input_ids = tokenizer(case1, return_tensors="pt").input_ids.to("cuda")
global_attention_mask = torch.zeros_like(input_ids)
global_attention_mask[:, 0] = 1

In [ ]:
input_ids

tensor([[   0,  133,   82,  ..., 7223,    4,    2]], device='cuda:0')

In [ ]:
sequences = model.generate(input_ids, global_attention_mask=global_attention_mask).sequences
summary = tokenizer.batch_decode(sequences, skip_special_tokens=True)

In [ ]:
standard_summary = sum1
system_summary = summary[0]

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, system_summary.split('.')[0])
pd.DataFrame(score[0]).set_index([['recall','precision','f-measure']])

,rouge-1,rouge-2,rouge-l
recall,1.000000,1.000000,1.000000
precision,0.666667,0.592857,0.666667
f-measure,0.800000,0.744395,0.800000


Test DataSet

In [ ]:
CasesText = test_cases['text']
GoldSummary = test_cases['summary']

In [ ]:
len(CasesText), len(GoldSummary)

(1237, 1237)

In [ ]:
# CasesText[:500]
# CasesText[500:800]
# CasesText[800:]

In [ ]:
SystemSummary = []

for i, case in enumerate(CasesText[800:]):
    
    input_ids = tokenizer(case, return_tensors="pt").input_ids.to("cuda")
    global_attention_mask = torch.zeros_like(input_ids)
    global_attention_mask[:, 0] = 1

    sequences = model.generate(input_ids, global_attention_mask=global_attention_mask).sequences
    Summary = tokenizer.batch_decode(sequences, skip_special_tokens=True)

    SystemSummary.append(Summary)
    print(i)

In [ ]:
SystemSummaryFinal = []

for i in SystemSummary:
  SystemSummaryFinal.append((i[0]))

In [ ]:
len(SystemSummaryFinal)

437

In [ ]:
Summaries = pd.DataFrame(list(zip(GoldSummary[800:], SystemSummaryFinal)), columns =['GoldSummary', 'SystemSummary'])

In [ ]:
path = "/content/drive/MyDrive/LegSuM/Data/Data by Models/"

Summaries.to_csv(path + "led-large3.csv", header=True, index=False)

**BART Model Score on Test**

In [ ]:
path = "/content/drive/MyDrive/LegSuM/Data/Data by Models/"

In [ ]:
file1 = pd.read_csv(path + "led-large1.csv")
file2 = pd.read_csv(path + "led-large2.csv")
file3 = pd.read_csv(path + "led-large3.csv")

In [ ]:
file1.shape, file2.shape, file3.shape, 

((500, 2), (300, 2), (437, 2))

In [ ]:
LEDlarge = pd.concat([file1, file2, file3])

In [ ]:
LEDlarge.reset_index(inplace= True, drop=True)

In [ ]:
LEDlarge.sample(5)

,GoldSummary,SystemSummary
669,The Private Investigator Act provides for the ...,the people of the state of california do enac...
828,"Existing law, except as specified, prohibits a...",this article amends section 13332.19 of the G...
934,Existing law makes it unlawful for any person ...,the people of the state of california do enac...
1214,"Under existing law, upon the proclamation of a...",during a state of emergency or local emergenc...
1191,"Existing law, effective February 1, 2012, diss...",the people of the state of California find an...


In [ ]:
system_summary = LEDlarge['SystemSummary']
standard_summary = LEDlarge['GoldSummary']

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, standard_summary, avg=True)

In [ ]:
LEDlargeRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])

In [ ]:
LEDlargeRouge

,rouge-1,rouge-2,rouge-l
recall,0.240947,0.117016,0.215438
precision,0.406354,0.209360,0.366007
f-measure,0.277481,0.135270,0.248556


In [ ]:
path = "/content/drive/MyDrive/LegSuM/scores/"

LEDlargeRouge.to_csv(path + "LEDlargeRouge.csv", header=True)